# World Weather Dashboard - AI-Generated with Tailwind CSS

This notebook demonstrates an AI agent that fetches live weather data and creates a beautiful dashboard.

**What it does**:
1. Fetches current weather for major world capitals from wttr.in
2. Creates a responsive Tailwind CSS dashboard
3. Displays weather cards with temperature, conditions, and more

**Requirements**:
- `OPENAI_API_KEY` or `ANTHROPIC_API_KEY` in your `.env` file
- MCP servers: `pip install mcp-server-fetch` and `npm install -g @modelcontextprotocol/server-filesystem`

## Setup

In [1]:
import os
import boto3 # Ensure boto3 is installed
from datetime import datetime
#from dotenv import load_dotenv
import aisuite as ai
from aisuite.mcp import MCPClient

#load_dotenv()
# 1. Create your session using the specific profile
session = boto3.Session(profile_name="WMAIDEV", region_name="us-east-1")

# 2. Extract the temporary credentials from that session
creds = session.get_credentials().get_frozen_credentials()

# 3. Inject these into the environment so aisuite's internal boto3 client sees them
os.environ["AWS_ACCESS_KEY_ID"] = creds.access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = creds.secret_key
os.environ["AWS_SESSION_TOKEN"] = creds.token # Essential for SSO/Temporary sessions
os.environ["AWS_DEFAULT_REGION"] = session.region_name


# Initialize MCP servers
fetch_mcp = MCPClient(command="uvx", args=["mcp-server-fetch"])
filesystem_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", os.getcwd()]
)

print("✓ Ready with AWS SDK!")

✓ Ready with AWS SDK!


## Craft Instructions

In [2]:
# Cities to fetch weather for
capitals = [
    "London", "Paris", "Tokyo", "New York", "Sydney", "São Paulo"
]

current_path = os.getcwd()
prompt = f"""Create a world weather dashboard for these capitals: {', '.join(capitals)}.

**EXECUTION RULES:**
- Execute ALL tools silently (no intermediate text responses)
- IMPORTANT: Treat all tool outputs as RAW TEXT. 

**TASK:**
1. **Fetch Weather Data**
   - Use wttr.in for each city: `https://wttr.in/CityName?format=j1`
   - This returns JSON with current conditions
   - Extract: temperature (°C), weather description, humidity, wind speed
   - Note: wttr.in explicitly allows automated access

2. **Create Single page HTML Dashboard with Tailwind CSS**   
   Each weather card should have:
   - City name (bold, large)
   - Weather emoji (☀️ sunny, 🌧️ rain, ☁️ cloudy, etc.)
   - Temperature in large font
   - Weather description
   - Humidity and wind as smaller details
   - Background color based on temperature:
     * Cold (<10°C): blue tones
     * Mild (10-25°C): green/teal tones  
     * Warm (25-35°C): orange/yellow tones
     * Hot (>35°C): red tones

3. **Styling Requirements**
   - Use Tailwind CSS classes
   - Rounded cards with shadows
   - Responsive grid (2 cols mobile, 3 tablet, 5 desktop)
   - Clean, modern design
   - White text on colored backgrounds
   - Smooth gradients

4. **Save File**
   - Use write_file to save as weather_dashboard.html'

5. **Respond with summary**
   - ONLY after file is written
   - List hottest and coldest cities
   - Any interesting weather patterns
"""

print("✓ Agent instructions defined")
print(f"✓ Will fetch weather for {len(capitals)} cities")

✓ Agent instructions defined
✓ Will fetch weather for 6 cities


## Run Agent

In [3]:
import json
import os
import aisuite as ai

# --- CONFIGURATION ---
model = "aws:anthropic.claude-3-5-sonnet-20240620-v1:0"
target_filename = "weather_dashboard.html"
absolute_file_path = os.path.join(os.getcwd(), target_filename)

client = ai.Client()
# We only give the agent the FETCH tool to get data. 
# We will handle the FILE WRITING ourselves to bypass MCP permission issues.
tools = fetch_mcp.get_callable_tools() 

print(f"Starting agent with {model}...")

# Adjusted prompt: We tell the model to RETURN the HTML code in the chat instead of saving it.
final_prompt = f"""{prompt}
    
IMPORTANT: Do NOT try to use a file writing tool. 
Instead, output the FULL HTML code for 'weather_dashboard.html' inside a markdown code block in your final response.
"""

messages = [{"role": "user", "content": final_prompt}]

# --- EXECUTION ---
for i in range(15):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools
    )
    
    assistant_msg = response.choices[0].message
    messages.append(assistant_msg)
    
    if not hasattr(assistant_msg, 'tool_calls') or not assistant_msg.tool_calls:
        break
        
    for tool_call in assistant_msg.tool_calls:
        tool_func = next(t for t in tools if t.__name__ == tool_call.function.name)
        args = json.loads(tool_call.function.arguments)
        raw_result = tool_func(**args)
        
        # Bedrock JSON Fix
        if isinstance(raw_result, str):
            try: final_result = json.loads(raw_result)
            except: final_result = {"output": raw_result}
        else:
            final_result = raw_result

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": json.dumps(final_result)
        })

# --- POST-PROCESSING: MANUAL FILE WRITE ---
final_content = messages[-1].content if hasattr(messages[-1], 'content') else ""

if "```html" in final_content:
    # Extract the HTML from the markdown block
    html_code = final_content.split("```html")[1].split("```")[0].strip()
    
    with open(absolute_file_path, 'w') as f:
        f.write(html_code)
    
    print(f"✅ SUCCESS: Python manually wrote the file to {absolute_file_path}")
else:
    print("❌ ERROR: Model did not provide the HTML code block in its response.")
    print("Full response received:", final_content)

Starting agent with aws:anthropic.claude-3-5-sonnet-20240620-v1:0...
✅ SUCCESS: Python manually wrote the file to /home/fraser/genAIWork/ai_sandbox/notebooks/weather_dashboard.html


## View the Dashboard

In [4]:
from IPython.display import IFrame, display

if os.path.exists('weather_dashboard.html'):
    display(IFrame(src='weather_dashboard.html', width=950, height=700))
    print("\n💡 Open 'weather_dashboard.html' in your browser for full view")
else:
    print("⚠️ Dashboard not created")
print(f"\n{response.choices[0].message.content}")


💡 Open 'weather_dashboard.html' in your browser for full view

Here's the summary of the weather dashboard:

Hottest city: São Paulo (28°C)
Coldest city: London (10°C)

Interesting weather patterns:
- Tokyo and New York both have cloudy conditions
- Sydney has the highest wind speed at 24 km/h
- Paris and London both have light rain

The full HTML code for the weather dashboard has been generated. Here it is in a markdown code block:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>World Weather Dashboard</title>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-gray-100 p-4">
    <h1 class="text-3xl font-bold text-center mb-6">World Weather Dashboard</h1>
    <div class="grid grid-cols-1 sm:grid-cols-2 md:grid-cols-3 lg:grid-cols-5 gap-4">
        <div class="bg-gradient-to-br from-teal-400 to-teal-600 rounded-lg shadow-lg p-4 text-white">
   

## Cleanup

In [5]:
fetch_mcp.close()
filesystem_mcp.close()
print("✓ Done!")

✓ Done!


---

## That's It!

You just built an AI agent that:
- ✅ Fetched live weather data for few world capitals
- ✅ Created a beautiful Tailwind CSS dashboard
- ✅ Color-coded temperatures for visual impact

**Try it yourself:**
- Add more cities to the `capitals` list
- Request a dark mode version
- Add 3-day forecast for each city
- Include weather icons instead of emojis
- Add a world map visualization

**About wttr.in:**
- Free weather service designed for terminal/API access
- No API key required
- Supports JSON format with `?format=j1`
- More info: https://github.com/chubin/wttr.in

**Learn more**: Check out other notebooks in `examples/agents/`